In [7]:
import folium

m = folium.Map(location=(-22.9140008, -43.563634))

In [8]:
m

In [9]:
m.save("html/index.html")

In [10]:
m = folium.Map(location=(-22.9140008, -43.563634), tiles="cartodb positron")
m

In [2]:
import folium, googlemaps, time
from folium.plugins import HeatMap, MarkerCluster, HeatMapWithTime
import pandas as pd

# carregando a chave api
import dotenv, os
%load_ext dotenv
chave_api = os.getenv("API_KEY")

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [12]:
client = googlemaps.Client(chave_api)

In [13]:
endereco = "rio de janeiro, rj, brasil"
coords_endereco = client.geocode(endereco)[0]["geometry"]["location"]

coords_endereco
# {'lat': -22.9068467, 'lng': -43.1728965}

{'lat': -22.9068467, 'lng': -43.1728965}

In [37]:
import json
def salva_resposta(nome_arquivo, resposta):
    dicionario = resposta

    with open(nome_arquivo, "w") as arquivo:
        json.dump(dicionario, arquivo)

In [16]:
# fazendo uma requisicao para todos os Mc Donalds proximos dentro do raio de distancia definido
# iterando todas as possiveis paginas de resposta
lista_coordenadas = []
keyword = "Mc Donalds"
response = client.places(query=keyword, location=coords_endereco, open_now=True)

for result in response["results"]:
        latitude = loja["geometry"]["location"]["lat"]
        longitude = loja["geometry"]["location"]["lng"]
        lista_coordenadas.append([latitude, longitude])

lista_coordenadas[0]
# [-22.9137805, -43.1666602]


In [48]:
mapa = folium.Map(location= lista_coordenadas[0], zoom_start= 13)
marker = MarkerCluster().add_to(mapa)

MarkerCluster(locations=lista_coordenadas).add_to(mapa)
mapa.save('mcdonalds_nearby_marker.html')
mapa

In [52]:
folium.Marker(
    location=[-22.9068467, -43.1728965],
    tooltip="Rio de Janeiro",
    popup="Informações",
    icon=folium.Icon(icon="user", color="red"),
).add_to(mapa)
mapa

In [53]:
lista_dfs = []
keyword = "Mc Donalds"
response = client.places_nearby(location=coords_endereco, keyword=keyword, radius=50000)

while True:
    time.sleep(0.1)

    for store in response["results"]:
        df = pd.DataFrame(
            {
                "nome": store["name"],
                "endereco": store["vicinity"],
                "latitude": store["geometry"]["location"]["lat"],
                "longitude": store["geometry"]["location"]["lng"],
            },
            index=[0],
        )
        lista_dfs.append(df)

    if "next_page_token" in response:
        next_page_token = response["next_page_token"]
        time.sleep(2.3)
        response = client.places_nearby(
            location=coords_endereco,
            keyword=keyword,
            radius=50000,
            page_token=next_page_token,
        )

    else:
        break

df_resultados = pd.concat(lista_dfs, ignore_index=True)
df_resultados.sample(5)

,nome,endereco,latitude,longitude
59,McDonald's,"Av. Monsenhor Félix, 1180 - Irajá, Rio de Janeiro",-22.832912,-43.327230
5,McDonald's,"Av. Ataulfo de Paiva, 406 - Leblon, Rio de Jan...",-22.983662,-43.219819
34,McDonald's,"Av. Quintino Bocaiúva, 61 - São Francisco, Nit...",-22.914953,-43.094791
20,MC Donald's,"Aeroporto Santos Dumont, 262 - Centro, Rio de ...",-22.912416,-43.166688
10,Mc donald's,"Av. das Américas, 19019 - Recreio dos Bandeira...",-23.021557,-43.488620


In [57]:
lojas_mc_filtrado = df_resultados[
    (df_resultados["nome"].str.lower().str.contains("mcdonald"))
    | df_resultados["nome"].str.lower().str.contains("mc donald")
]
lojas_mc_filtrado = lojas_mc_filtrado.drop_duplicates(subset="endereco")

lista_coordenadas = lojas_mc_filtrado[['latitude', 'longitude']].values

print('Quantidade de resultados:', len(lista_coordenadas))
# Quantidade de resultados: 60
print('Exemplo de resultado:', lista_coordenadas[0])
# Exemplo de resultado: [-22.9056077 -43.1761831]

Quantidade de resultados: 60
Exemplo de resultado: [-22.9056077 -43.1761831]


In [60]:
map_calor = folium.Map(location=list(coords_endereco.values()), zoom_start=11, tiles="Cartodb dark_matter",)
HeatMap(lista_coordenadas, radius= 30).add_to(map_calor)
map_calor.save('mcdonalds_heatmap.html')
map_calor

# Renner

In [66]:
df = pd.read_excel(
    "Planilhas e Fundamentos .xlsm", 
    sheet_name= 'Lista de Lojas | Stores List', 
    skiprows= 5,
    usecols = ['Opening date', 'Country', 'UF', 'State', 'City'])

df_renner = df.copy()
df_renner

,Opening date,Country,UF,State,City
0,2023-10-17 00:00:00,Brazil,MG,Minas Gerais,Ituiutaba
1,2023-10-11 00:00:00,Brazil,SC,Santa Catarina,Videira
2,2023-09-20 00:00:00,Brazil,ES,Espírito Santo,Vitória
3,2023-09-15 00:00:00,Brazil,PB,Paraíba,João Pessoa
4,2023-09-09 00:00:00,Brazil,GO,Goiás,Goiânia
...,...,...,...,...,...
764,41,4,57,NaN,NaN
765,118,46,205,NaN,NaN
766,10,0,10,NaN,NaN
767,4,0,4,NaN,NaN


In [70]:
# removendo dados faltantes
df_renner.dropna(inplace= True)

# tratando a data de abertura
df_renner['Opening date'] = pd.to_datetime(df_renner['Opening date']) 

# filtrando o pais
df_renner = df_renner[df_renner['Country'] == 'Brazil']

# criando uma coluna com endereco
df_renner['Address'] = df_renner.apply(lambda row: f"{row['City']} - {row['State']} - {row['Country']}", axis= 1)

df_renner.head(5)

,Opening date,Country,UF,State,City,Address
0,2023-10-17,Brazil,MG,Minas Gerais,Ituiutaba,Ituiutaba - Minas Gerais - Brazil
1,2023-10-11,Brazil,SC,Santa Catarina,Videira,Videira - Santa Catarina - Brazil
2,2023-09-20,Brazil,ES,Espírito Santo,Vitória,Vitória - Espírito Santo - Brazil
3,2023-09-15,Brazil,PB,Paraíba,João Pessoa,João Pessoa - Paraíba - Brazil
4,2023-09-09,Brazil,GO,Goiás,Goiânia,Goiânia - Goiás - Brazil


In [72]:
def get_coords(address): return client.geocode(address)[0]['geometry']['location']
df_renner['Coords'] = df_renner['Address'].apply(lambda coords: tuple(get_coords(coords).values()))

df_renner_copia = df_renner.copy()
df_renner.head(4)

,Opening date,Country,UF,State,City,Address,Coords
0,2023-10-17,Brazil,MG,Minas Gerais,Ituiutaba,Ituiutaba - Minas Gerais - Brazil,"(-18.9745738, -49.4600885)"
1,2023-10-11,Brazil,SC,Santa Catarina,Videira,Videira - Santa Catarina - Brazil,"(-27.0052075, -51.15439500000001)"
2,2023-09-20,Brazil,ES,Espírito Santo,Vitória,Vitória - Espírito Santo - Brazil,"(-20.3196644, -40.3384748)"
3,2023-09-15,Brazil,PB,Paraíba,João Pessoa,João Pessoa - Paraíba - Brazil,"(-7.118835199999999, -34.8814339)"
4,2023-09-09,Brazil,GO,Goiás,Goiânia,Goiânia - Goiás - Brazil,"(-16.6868912, -49.2647943)"
...,...,...,...,...,...,...,...
724,1980-09-01,Brazil,RS,Rio Grande do Sul,Santa Maria,Santa Maria - Rio Grande do Sul - Brazil,"(-29.68949839999999, -53.7923441)"
725,1977-11-01,Brazil,RS,Rio Grande do Sul,Porto Alegre,Porto Alegre - Rio Grande do Sul - Brazil,"(-30.0368176, -51.2089887)"
726,1976-04-10,Brazil,RS,Rio Grande do Sul,Canoas,Canoas - Rio Grande do Sul - Brazil,"(-29.9077898, -51.1847036)"
727,1970-12-04,Brazil,RS,Rio Grande do Sul,Porto Alegre,Porto Alegre - Rio Grande do Sul - Brazil,"(-30.0368176, -51.2089887)"


In [5]:
# df_renner = df_renner_copia.copy()
# df_renner.to_csv('df_renner-coords.csv', index=False)
df_renner = pd.read_csv('DataFrames/df_renner-coords.csv')
df_renner['Opening date'] = pd.to_datetime(df_renner['Opening date']) 


,Opening date,Country,UF,State,City,Address,Coords
0,2023-10-17,Brazil,MG,Minas Gerais,Ituiutaba,Ituiutaba - Minas Gerais - Brazil,"(-18.9745738, -49.4600885)"
1,2023-10-11,Brazil,SC,Santa Catarina,Videira,Videira - Santa Catarina - Brazil,"(-27.0052075, -51.15439500000001)"
2,2023-09-20,Brazil,ES,Espírito Santo,Vitória,Vitória - Espírito Santo - Brazil,"(-20.3196644, -40.3384748)"
3,2023-09-15,Brazil,PB,Paraíba,João Pessoa,João Pessoa - Paraíba - Brazil,"(-7.118835199999999, -34.8814339)"
4,2023-09-09,Brazil,GO,Goiás,Goiânia,Goiânia - Goiás - Brazil,"(-16.6868912, -49.2647943)"
...,...,...,...,...,...,...,...
709,1980-09-01,Brazil,RS,Rio Grande do Sul,Santa Maria,Santa Maria - Rio Grande do Sul - Brazil,"(-29.68949839999999, -53.7923441)"
710,1977-11-01,Brazil,RS,Rio Grande do Sul,Porto Alegre,Porto Alegre - Rio Grande do Sul - Brazil,"(-30.0368176, -51.2089887)"
711,1976-04-10,Brazil,RS,Rio Grande do Sul,Canoas,Canoas - Rio Grande do Sul - Brazil,"(-29.9077898, -51.1847036)"
712,1970-12-04,Brazil,RS,Rio Grande do Sul,Porto Alegre,Porto Alegre - Rio Grande do Sul - Brazil,"(-30.0368176, -51.2089887)"


In [4]:
# criando uma coluna com o ano
df_renner['Opening Year'] = df_renner['Opening date'].dt.year

# removendo dados desnecessários
df_renner = df_renner.drop(['Country', 'UF', 'State', 'City', 'Opening date', 'Address'], axis=1)

# Ordenando pelo ano
df_renner = df_renner.sort_values(by='Opening Year')

df_renner.head(5)

AttributeError: Can only use .dt accessor with datetimelike values

In [101]:
# criando uma timeline do primeiro ao ultimo ano
primeiro_ano = df_renner['Opening Year'].min()
ultimo_ano = df_renner['Opening Year'].max()
years_time_line = list(range(primeiro_ano, ultimo_ano))
years_time_line.append(ultimo_ano)

data_renner = []
for year in years_time_line:
    lojas_coordenadas = []
    for index, row in df_renner.iterrows():
        # calcula o tempo que a loja está aberta
        year_diff =  year - row['Opening Year']


        print("year_diff", year_diff,'ano abertura', row['Opening Year'] )
        # caso seja menor que zero, não é adicionada ao mapa
        if year_diff < 0:
            pass
        else:
            print('pass')
            coord = [row['Coords'][0], row['Coords'][1]]
            if year_diff == 0: coord.append(0.2)
            if year_diff == 1: coord.append(0.5)
            elif year_diff == 2: coord.append(0.7)
            elif year_diff > 2: coord.append(1)
            
            year_coords.append(coord)
    
    data_renner.append(year_coords)


year_diff 0 ano abertura 1967
pass
year_diff -3 ano abertura 1970
year_diff -9 ano abertura 1976
year_diff -10 ano abertura 1977
year_diff -13 ano abertura 1980
year_diff -16 ano abertura 1983
year_diff -18 ano abertura 1985
year_diff -20 ano abertura 1987
year_diff -20 ano abertura 1987
year_diff -24 ano abertura 1991
year_diff -26 ano abertura 1993
year_diff -27 ano abertura 1994
year_diff -28 ano abertura 1995
year_diff -29 ano abertura 1996
year_diff -29 ano abertura 1996
year_diff -29 ano abertura 1996
year_diff -30 ano abertura 1997
year_diff -30 ano abertura 1997
year_diff -30 ano abertura 1997
year_diff -31 ano abertura 1998
year_diff -31 ano abertura 1998
year_diff -31 ano abertura 1998
year_diff -31 ano abertura 1998
year_diff -31 ano abertura 1998
year_diff -31 ano abertura 1998
year_diff -32 ano abertura 1999
year_diff -32 ano abertura 1999
year_diff -32 ano abertura 1999
year_diff -32 ano abertura 1999
year_diff -32 ano abertura 1999
year_diff -32 ano abertura 1999
year_di

In [102]:
mapa2 = folium.Map(location= [-17, -50], zoom_start=4)
HeatMap = HeatMapWithTime(data_renner, index=years_time_line, auto_play=True, max_opacity=0.3, name='Datas', blur=1, min_speed= 1)
# HeatMap = HeatMapWithTime(data, index=time_index, auto_play=True, max_opacity=0.3)
HeatMap.add_to(mapa2)
mapa2